In [1]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ModuleNotFoundError: No module named 'torch'

In [ ]:
import os
video_directory = "/home/ankitk/Desktop/AVQA/MUSIC-AVQA-videos-Real-002/MUSIC-AVQA-videos-Real"  # Replace with the actual directory path
video_file=[]
for index, row in train_df.iterrows():
    video_id = row['video_id']
    video_file_path = os.path.join(video_directory, f"{video_id}.mp4")
    video_file.append(video_file_path)
#     print(video_file_path)

In [5]:
len(video_file)

10000

In [6]:
train_df["video_id"].nunique()

5724

In [5]:
#### VideoEmbeddings

In [ ]:
import av
import torch
import numpy as np

from transformers import AutoImageProcessor, VideoMAEModel
from huggingface_hub import hf_hub_download

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def extract_video_features(file_path, device):
    container = av.open(file_path)

    indices = sample_frame_indices(clip_len=16, frame_sample_rate=1, seg_len=container.streams.video[0].frames)
    video = read_video_pyav(container, indices)

    image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base")
    model = VideoMAEModel.from_pretrained("MCG-NJU/videomae-base")
    model.to(device)

    inputs = image_processor(list(video), return_tensors="pt")
    inputs = inputs.to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        last_hidden_states = outputs.last_hidden_state

    features = last_hidden_states.mean(dim=1).squeeze().to("cpu").numpy()
    
    return features

video_files = video_file
all_features = []
count = 0
for file_path in video_files:
    features = extract_video_features(file_path, device)
    print(f"Features for {file_path}:")
    print(features.shape)  
    all_features.append(features)
    
all_features_array = np.array(all_features)

np.save("./video_features.npy", all_features_array)

print("Processing completed for all videos")

## Audio Embeddings

In [ ]:
import os
import torchaudio
import torchaudio.transforms
import torch
import numpy as np
from transformers import VideoMAEImageProcessor, VideoMAEModel, ASTFeatureExtractor, ASTModel


audio_folder = "/home/ankitk/Desktop/AVQA/Audio_wav"

embeddings_folder = "/home/ankitk/Desktop/AVQA/Audio_features"
os.makedirs(embeddings_folder, exist_ok=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
audio_processor = ASTFeatureExtractor()
audio_model = ASTModel.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593").to(device)


for filename in os.listdir(audio_folder):
    if filename.endswith(".wav"):
        audio_segment_path = os.path.join(audio_folder, filename)

        stereo_audio, sample_rate = torchaudio.load(audio_segment_path)

        mono_audio = stereo_audio.mean(dim=0)

        resample = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
        mono_audio_16khz = resample(mono_audio)

        inputs = audio_processor(mono_audio_16khz, sampling_rate=16000, return_tensors="pt").to(device)

        with torch.no_grad():
            outputs = audio_model(**inputs)

        last_hidden_states = outputs.last_hidden_state.mean(dim=1).squeeze()

        embedding_path = os.path.join(embeddings_folder, f"{filename[:-4]}.npy")
        np.save(embedding_path, last_hidden_states.cpu().numpy())

        print(f"Audio embedding saved for {filename}")

print("All audio embeddings saved.")